In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vstepanenko/disaster-tweets")

print("Path to dataset files:", path)

100%|██████████| 656k/656k [00:00<00:00, 79.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/vstepanenko/disaster-tweets/versions/3


In [ ]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/vstepanenko/disaster-tweets/versions/3/tweets.csv')

In [ ]:
def get_tokens(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)          # Remove punctuation/numbers
    tokens = [w for w in text.split() if w not in stop_words and len(w) > 3]
    return list(set(tokens))

In [ ]:
transactions = df['text'].head(2000).apply(get_tokens).tolist()

In [ ]:
def eclat(data, min_support, max_length):
    # Step A: Convert to Vertical Format (Word -> Set of Transaction IDs)
    vertical_data = {}
    for tid, transaction in enumerate(data):
        for item in transaction:
            if item not in vertical_data:
                vertical_data[item] = set()
            vertical_data[item].add(tid)

    # Filter items that meet minimum support
    limit = len(data) * min_support
    frequent_itemsets = {tuple([item]): tids for item, tids in vertical_data.items() if len(tids) >= limit}

In [ ]:
def eclat(data, min_support, max_length):
    # Step A: Convert to Vertical Format (Word -> Set of Transaction IDs)
    vertical_data = {}
    for tid, transaction in enumerate(data):
        for item in transaction:
            if item not in vertical_data:
                vertical_data[item] = set()
            vertical_data[item].add(tid)

    # Filter items that meet minimum support
    limit = len(data) * min_support
    frequent_itemsets = {tuple([item]): tids for item, tids in vertical_data.items() if len(tids) >= limit}

    # Step B: Recursive Intersections
    items = sorted(list(frequent_itemsets.keys()))

    def recurse(current_itemsets, level):
        if level > max_length:
            return

        new_itemsets = {}
        items_list = list(current_itemsets.keys())

        for i in range(len(items_list)):
            for j in range(i + 1, len(items_list)):
                # Join itemsets if they share the same prefix
                it1, it2 = items_list[i], items_list[j]
                if it1[:-1] == it2[:-1]:
                    combined_itemset = it1 + (it2[-1],)
                    # Intersect TID sets
                    intersection_tids = current_itemsets[it1].intersection(current_itemsets[it2])

                    if len(intersection_tids) >= limit:
                        new_itemsets[combined_itemset] = intersection_tids
                        frequent_itemsets[combined_itemset] = intersection_tids

        if new_itemsets:
            recurse(new_itemsets, level + 1)

    recurse(frequent_itemsets, 2)
    return frequent_itemsets

In [ ]:
min_sup = 0.01
results = eclat(transactions, min_support=min_sup, max_length=2)

In [ ]:
print(f"--- Frequent Word Associations (Support > {min_sup*100}%) ---")
# Filter for pairs (length 2) to see associations
associations = {k: len(v) for k, v in results.items() if len(k) > 1}
# Sort by frequency
sorted_associations = sorted(associations.items(), key=lambda x: x[1], reverse=True)

for words, count in sorted_associations[:15]:
    print(f"Words: {words} | Frequency: {count}")

--- Frequent Word Associations (Support > 1.0%) ---
Words: ('fires', 'bush') | Frequency: 45
Words: ('body', 'bags') | Frequency: 42
Words: ('burning', 'buildings') | Frequency: 34
Words: ('accident', 'airplane') | Frequency: 32
Words: ('fire', 'buildings') | Frequency: 29
Words: ('need', 'battle') | Frequency: 24
Words: ('body', 'bagging') | Frequency: 24
Words: ('need', 'backup') | Frequency: 23
Words: ('battle', 'backup') | Frequency: 23
Words: ('bridge', 'collapse') | Frequency: 23
Words: ('body', 'parts') | Frequency: 21


In [ ]:
print("\n--- Associations with the word 'fire' ---")
for words, count in associations.items():
    if 'body' in words:
        print(f"{words}: {count}")


--- Associations with the word 'fire' ---
('body', 'parts'): 21
('body', 'bags'): 42
('body', 'bagging'): 24
